### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import json
import re
from collections import Counter, namedtuple

### 載入資料

In [3]:
with open('/content/drive/MyDrive/NLP/WebNews.json', 'r') as f:
    news_data = json.load(f)

In [4]:
news_data[0].keys()

dict_keys(['file', 'link', 'id', 'istop', 'img', 'subject', 'datasourceunit', 'liaisonemail', 'fsort', 'detailcontent', 'secsubject', 'closedate', 'liaisonfax', 'postdate', 'postunit', 'liaisonper', 'liaisontel', 'topdate'])

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [5]:
#取出新聞內文
corpus_list = [news_data[n]['detailcontent'] for n in range(len(news_data))]

#去除HTML Tags與標點符號(只保留英文、數字、中文)
corpus_list = [re.sub('<.*?>|&nbsp;|[\(\)（）、，。「」：；『』%‧,\^\$\*\s\-_&@\!\?]','',corpus_list[n]) for n in range(len(corpus_list))]

corpus_list[0]

'迎接國旅爆發期五星縣市長合體推廣桃園屏東互惠旅遊桃園市長鄭文燦今11日下午出席桃園加屏東旅遊優惠強棒一棒接一棒好康發表記者會鄭市長表示隨著疫情趨緩國內進入國旅爆發期觀光局近期公布最新10大熱門旅遊景點桃園包辦前三名依序為大溪老街石門水庫及角板山顯示桃園旅遊的潛力無限鄭市長說桃園市與屏東縣各自擁有獨特的觀光魅力與資源希望能搭配中央觀光振興方案共同推廣防疫互惠旅遊並推出更多加碼優惠措施降低民眾的旅遊負擔增加旅遊的樂趣也讓國境之南與國境之門做最好的交流鄭市長提到桃園是國門之都屏東是國境之南兩個城市各有特色及魅力許多景點也互相輝映假如你喜歡屏東的山川琉璃吊橋絕對不能錯過桃園的新溪口吊橋如果你喜歡大溪老街也應該去恆春古城漫遊鄭市長說許多民眾造訪過很多國外景點例如普吉島巴里島卻沒有到過小琉球澎湖等離島十分可惜他也稱讚屏東幅員遼闊擁有多樣化的景點墾丁更是國內旅遊勝地之一希望現階段推動防疫新生活的同時桃園也能與各縣市合作一起推廣防疫互惠旅遊讓更多在地旅遊業者受惠桃園預估下半年包含鄰里長環保志工守望相助隊義警民防等觀摩旅遊團數出團數將達500團以上目前桃園第一階段已經與基隆台南屏東等縣市簽署合作契約第二階段將與宜蘭花蓮南投嘉義等縣市合作希望未來逐漸擴大至其他縣市提供遊客更多優惠措施鄭市長也說屏東縣推出三鐵優惠方案桃園市自7月1日起推出來桃園住一晚加贈免費一日遊方案遊客來桃園在合法旅宿住一晚即可獲得市府加贈免費一日遊行程內容包含遊覽車接送午餐景點門票或DIY體驗等期盼帶領遊客體驗桃園的觀光魅力此外民眾也可以持200元振興三倍券兌換400元夜市券藉此鼓勵民眾到夜市消費市府未來也將陸續推出更多的加碼優惠措施屏東縣長潘孟安則希望在鄭市長推廣下吸引更多桃園市民到屏東旅遊他表示桃園為國境之門擁有很大的發展優勢在鄭市長帶領下桃園連續多年結婚率及出生率都是全國之冠也成為一座宜居城市他也特別推薦親子旅遊首選屏東屏東縣政府整合轄內旅宿業者伴手禮業者食品業者等推出鐵定加碼鐵定貼心鐵定好玩三鐵優惠7月15日以前遊客在恆春地區住宿一晚即加贈場館門票或等值餐飲券在東港地區住宿則可用10元加購伴手禮及小王馬福袋10人以上團體旅遊每人可獲得500元旅費補助每團上限1萬元屏東也有許多當季優質農特產品及特色伴手禮歡迎桃園市民踴躍到屏東旅遊保證遊客玩得痛快買得愉快吃得爽快全國中等學校運動會今109年首度移師屏東舉辦潘縣

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [7]:
def ngram(documents, N=2):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率
    for doc in documents:
        # 在每個文章前加上起始(<start>)與結束符號(<end>)
        doc = ['<s>']+list(doc)+['<e>']
        # 計算分子[(word*n)()...] => N個詞一組
        [total_grams.append(tuple(doc[i:i+N])) for i in range(len(doc)-N+1)]
        # 計算分母 => N-1個詞一組
        [words.append(tuple(doc[i:(i+N-1)])) for i in range(len(doc)-N+2)]

    #計算分子詞頻 
    total_word_counter = Counter(total_grams)
    #計算分母詞頻
    word_counter = Counter(words)

    #計算所有N-gram預測字詞的機率
    for key in total_word_counter:
        word_1=''.join(key[:N-1])
        if word_1 not in ngram_prediction:
            ngram_prediction[word_1]=set()
        prob = total_word_counter[key]/word_counter[key[:N-1]]
        ngram_prediction[word_1].add(Word(key[-1],prob))  #set.add()

    return ngram_prediction    

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [8]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(corpus_list, N=4)
for word, pred in four_gram_pred.items():
    four_gram_pred[word]=sorted(pred,key=lambda x:x.prob,reverse=True)

In [ ]:
# 執行時因為字典數龐大，顯示會較慢
four_gram_pred

In [10]:
#給定字詞，使用ngram預測下一個字的機率(顯示top 10)
text = '鄭文燦'
next_words = four_gram_pred['鄭文燦']
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 今, probability: 0.9795698924731183
next word: 市, probability: 0.00967741935483871
next word: 模, probability: 0.002150537634408602
next word: 表, probability: 0.002150537634408602
next word: 昨, probability: 0.001075268817204301
next word: 台, probability: 0.001075268817204301
next word: 率, probability: 0.001075268817204301
next word: 於, probability: 0.001075268817204301
next word: 成, probability: 0.001075268817204301
next word: 回, probability: 0.001075268817204301


可自行嘗試使用不同的N搭建不同的N-Gram模型的效果